In [ ]:
#include "MGWFErfFit.hh"
#include <cmath>

using namespace std;

MGWFErfFit::MGWFErfFit(const string& aName) : 
MGVWaveformParameterCalculator(aName),
fDoBaselineRemoval(false), 
fNSamplesToIntegrate(1), 
fNSigmaToFit(0), 
fNIterMax(10),
fMuMin(0), 
fMuMax(0), 
fSigMin(0), 
fSigMax(0), 
fMuTol(0.1), 
fSigTol(0),
fNStepsToRamp(5), 
fDoDebug(false)
{
    AddParameter("mu");
    AddParameter("sigma");
    AddParameter("chi2");
    AddParameter("nIter");
    AddParameter("flags");
}

void MGWFErfFit::CalculateParameters(const MGWaveform& wf) 
    {
        fWf = wf;

        if(fNSamplesToIntegrate > fWf.GetLength()) 
        fNSamplesToIntegrate = fWf.GetLength();

        if(fNSamplesToIntegrate < 1) 
        fNSamplesToIntegrate = 1;

        #baseline removal
        double baseline = 0;
        if(fDoBaselineRemoval) 
        {
            for(size_t i=0; i<fNSamplesToIntegrate; i++) baseline += fWf[i];
            baseline /= double(fNSamplesToIntegrate);
        }

        #normalizing the wf between [0:1]
        double amp = 0;
        for(size_t i=0; i<fNSamplesToIntegrate; i++) 
            {
                amp += fWf[fWf.GetLength()-1-i];
            }
        amp /= double(fNSamplesToIntegrate);

        fWf -= baseline;
        if(amp != 0) 
        fWf /= amp;
        #-------------
        
        #start fitting
        double iMu = ParVal(kMu)*fWf.GetSamplingFrequency();
        double iSig = ParVal(kSigma)*fWf.GetSamplingFrequency();
        if(iSig == 0) 
        iSig = 1./(fWf[size_t(iMu) + 5] - fWf[size_t(iMu)-5])*10/sqrt(2.*3.14159265359);
        if(iSig == 0) 
        {
            cout << "MGWFErfFit: Must start with non-zero sigma." << endl;
            return;
        }
        if(fDoDebug) cout << "guess: " << iMu << ' ' << iSig << endl;

        size_t iMin = 0;
        size_t iMax = fWf.GetLength()-1;
        double iMuLast = iMu;
        double iSigLast = iSig;
        double iMuMin = fMuMin*fWf.GetSamplingFrequency();
        double iMuMax = fMuMax*fWf.GetSamplingFrequency();
        double iSigMin = fSigMin*fWf.GetSamplingFrequency();
        double iSigMax = fSigMax*fWf.GetSamplingFrequency();
        double iMuTol = fMuTol*fWf.GetSamplingFrequency();
        double iSigTol = fSigTol*fWf.GetSamplingFrequency();
        double chi2 = 0;
        uint32_t flags = 0;
        size_t iIter;
        for(iIter = 0; iIter < fNIterMax; iIter++) 
            {
                if(fNSigmaToFit != 0) 
                {
                    iMin = iMu > fNSigmaToFit*iSig ? size_t(iMu - fNSigmaToFit*iSig) : 0;
                    iMax = size_t(iMu + fNSigmaToFit*iSig);
                    if(iMax >= fWf.GetLength()) 
                    iMax = fWf.GetLength()-1;
                }

                double dg = 0, dgx = 0, dgx2 = 0, dgx3 = 0, g2 = 0, g2x = 0, g2x2 = 0;
                chi2 = 0;
                for(size_t i=iMin; i<=iMax; i++) 
                {
                    double x = (-iMu + i)/iSig;
                    double f = 0.5*(1. + erf(x/sqrt(2.)));
                    double d = fWf[i] - f;
                    double g = exp(-x*x/2.);
                    dg += d*g;
                    dgx += d*g*x;
                    dgx2 += d*g*x*x;
                    dgx3 += d*g*x*x*x;
                    g2 += g*g;
                    g2x += g*g*x;
                    g2x2 += g*g*x*x;
                    chi2 += d*d;
                }
                double gnorm = 1./sqrt(2.*3.14159265359)/iSig;
                g2 *= gnorm;
                g2x *= gnorm;
                g2x2 *= gnorm;

                double H11 = g2 + dgx/iSig;
                double H12 = g2x + dgx2/iSig - dg/iSig;
                double H22 = g2x2 + dgx3/iSig - 2.*dgx/iSig;
                double detH = H11*H22 - H12*H12;

                double stepDownFactor = 1.;
                if(iIter < fNStepsToRamp) 
                    stepDownFactor = 1 + fNStepsToRamp - iIter;
                iMu -= (H22*dg - H12*dgx)/detH/stepDownFactor;
                if(iMu < iMuMin) iMu = iMuMin;
                if(iMu >= iMuMax) iMu = iMuMax;
                iSig -= (-H12*dg + H11*dgx)/detH/stepDownFactor;
                if(iSig < iSigMin) iSig = iSigMin;
                if(iSig > iSigMax) iSig = iSigMax;
                if(fDoDebug) cout << iIter+1 << ' ' << iMu << ' ' << iSig << endl;

                if(fabs(iMu - iMuLast) < iMuTol && fabs(iSig - iSigLast) < iSigTol) 
                    break;
                iMuLast = iMu;
                iSigLast = iSig;

                if(iIter == fNIterMax-1) 
                    {
                      flags |= (0x1 << kIterAtMax);
                      if(fDoDebug) cout << "MGWFErfFit: went to fNIterMax" << endl;
                    }
            }

            if(iMu == iMuMin) 
            {
                flags |= (0x1 << kMuAtMin);
                if(fDoDebug) cout << "MGWFErfFit: mu at lower limit" << endl;
            }
            if(iMu == iMuMax) 
            {
                flags |= (0x1 << kMuAtMax);
                if(fDoDebug) cout << "MGWFErfFit: mu at upper limit" << endl;
            }
            if(iSig == iSigMin) 
            {
                flags |= (0x1 << kSigAtMin);
                if(fDoDebug) cout << "MGWFErfFit: sigma at lower limit" << endl;
            }
            if(iSig == iSigMax) 
            {
                flags |= (0x1 << kSigAtMax);
                if(fDoDebug) cout << "MGWFErfFit: sigma at upper limit" << endl;
            }
            if(amp == 0) 
            {
                flags |= (0x1 << kZeroAmp);
                if(fDoDebug) cout << "MGWFErfFit: amplitude was zero" << endl;
            }

            ParVal(kMu) = iMu*fWf.GetSamplingPeriod();
            ParVal(kSigma) = iSig*fWf.GetSamplingPeriod();
            ParVal(kChi2) = chi2;
            ParVal(kNIter) = iIter;
            ParVal(kFlags) = flags;
}